In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
## bUILT-iN pACKAGES
import pickle
from collections import defaultdict
import json
import numpy as np
import math
import matplotlib.pyplot as plt
np.set_printoptions(suppress=True)
%matplotlib inline
from IPython.display import display

## lAST sUMMER
from romtoslp import *
from sentences import *
from DCS import *
import MatDB

## lAST yEAR
# from word_definite import *
# from nnet import *
# from heap_n_PrimMST import *
# from word_definite import *

In [3]:
import winsound
def playBeep():
    for i in range(3):
        winsound.Beep(2200, 300)
        winsound.Beep(2600, 300)

In [4]:
loaded_SKT = pickle.load(open('../Simultaneous_CompatSKT_10K.p', 'rb'), encoding=u'utf-8')
loaded_DCS = pickle.load(open('../Simultaneous_DCS_10K.p', 'rb'), encoding=u'utf-8')

In [5]:
dataset_4k_1k = pickle.load(open('../SmallDataset_4K_1K.p', 'rb'))
TrainFiles = dataset_4k_1k['TrainFiles']
TestFiles = dataset_4k_1k['TestFiles']

In [6]:
matDB = MatDB.MatDB()

In [7]:
# from MatDB import *
import word_definite as WD
from heap_n_PrimMST import *
from nnet import *
"""
################################################################################################
######################  CREATE SEVERAL DATA STRUCTURES FROM SENTENCE/DCS  ######################
###########################  NODELIST, ADJACENCY LIST, GRAPH, HEAP #############################
################################################################################################
"""
def GetTrainingKit(sentenceObj, dcsObj):
    nodelist = GetNodes(sentenceObj)
    
    # Nodelist with only the correct_nodes
    nodelist2 = GetNodes(sentenceObj)
    nodelist2_to_correct_mapping = {}
    nodelist_correct = []
    search_key = 0
    first_key = 0
    for chunk_id in range(len(dcsObj.lemmas)):
        while nodelist2[first_key].chunk_id != chunk_id:
            first_key += 1
        for j in range(len(dcsObj.lemmas[chunk_id])):
            search_key = first_key
            while (nodelist2[search_key].lemma != rom_slp(dcsObj.lemmas[chunk_id][j])) or (nodelist2[search_key].cng != dcsObj.cng[chunk_id][j]):
                search_key += 1
                if search_key >= len(nodelist2) or nodelist2[search_key].chunk_id > chunk_id:
                    break
    #         print((rom_slp(dcsObj.lemmas[chunk_id][j]), dcsObj.cng[chunk_id][j]))
    #         print(nodelist[search_key])
            nodelist2_to_correct_mapping[len(nodelist_correct)] = search_key
            nodelist_correct.append(nodelist2[search_key])
    return (nodelist, nodelist_correct, nodelist2_to_correct_mapping)
    

def GetGraph(nodelist, neuralnet):
    if not neuralnet.outer_relu:
        conflicts_Dict = Get_Conflicts(nodelist)

        featVMat = Get_Feat_Vec_Matrix(nodelist, conflicts_Dict)

        (WScalarMat, SigmoidGateOutput) = Get_W_Scalar_Matrix_from_FeatVect_Matrix(featVMat, nodelist, conflicts_Dict, neuralnet)
        return (conflicts_Dict, featVMat, WScalarMat, SigmoidGateOutput)
    else:
        conflicts_Dict = Get_Conflicts(nodelist)

        featVMat = Get_Feat_Vec_Matrix(nodelist, conflicts_Dict)

        WScalarMat = Get_W_Scalar_Matrix_from_FeatVect_Matrix(featVMat, nodelist, conflicts_Dict, neuralnet)
        return (conflicts_Dict, featVMat, WScalarMat)

In [8]:
import time
st = str(int((time.time() * 1e6) % 1e13))
log_name = 'logs/train_nnet_t{}.out'.format(st)
p_name = 'outputs/train_nnet_t{}.p'.format(st)
print(p_name)
# outputs/train_nnet_t9543323920940.p - 80%

outputs/train_nnet_t9671431073874.p


In [9]:
"""
################################################################################################
##############################  MAIN FUNCTION  #################################################
################################################################################################
"""

def train(loaded_SKT, loaded_DCS, n_trainset = -1):
    # Train
    filePerBatch = 15
    iterationPerBatch = 5
    if n_trainset == -1:
        totalBatchToTrain = 20
    else:
        totalBatchToTrain = math.ceil(n_trainset/filePerBatch)
    
    for iterout in range(totalBatchToTrain):
        # Change current batch
        trainer.Save(p_name)
        print('Batch: ', iterout)
        files_for_batch = TrainFiles[iterout*filePerBatch:(iterout + 1)*filePerBatch]
        print(files_for_batch)
        # trainer.Load('outputs/neuralnet_trained.p')
        
        # Run few times on same set of files
        for iterin in range(iterationPerBatch):
            print('ITERATION IN', iterin)        
            for fn in files_for_batch:
                sentenceObj = loaded_SKT[fn]
                dcsObj = loaded_DCS[fn]
                # trainer.Save('outputs/saved_trainer.p')
                try:
                    trainer.Train(sentenceObj, dcsObj)
                except:
                    print('\x1b[31mFailed: {} \x1b[0m'.format(sentenceObj.sent_id))
    trainer.Save(p_name)
                
def test(loaded_SKT, loaded_DCS, n_testSet = -1, _testFiles = None):
    total_lemma = 0;
    correct_lemma = 0;

    total_word = 0;
    total_output_nodes = 0
    correct_word = 0;
    file_counter = 0
    if _testFiles is None:
        if n_testSet == -1:
            _testFiles = TestFiles
        else:
            _testFiles = TestFiles[0:n_testSet]
    else:
        if n_testSet == -1:
            _testFiles = _testFiles
        else:
            _testFiles = _testFiles[0:n_testSet]
    for fn in _testFiles:
        if file_counter % 100 == 0:
            print(file_counter,' Checkpoint... ')
        file_counter += 1
        sentenceObj = loaded_SKT[fn]
        dcsObj = loaded_DCS[fn]        
        try:
            (word_match, lemma_match, n_dcsWords, n_output_nodes) = trainer.Test(sentenceObj, dcsObj)
            total_lemma += n_dcsWords
            total_word += n_dcsWords
            
            total_output_nodes += n_output_nodes            
            
            correct_lemma += lemma_match
            correct_word += word_match
        except:
            print('Failed!')        

    print('Micro Recall[lemma]: {}'.format(100*correct_lemma/total_word))
    print('Micro Recall[word]: {}'.format(100*correct_word/total_word))
    
    print('Micro Precision[lemma]: {}'.format(100*correct_lemma/total_output_nodes))
    print('Micro Precision[word]: {}'.format(100*correct_word/total_output_nodes))
    

In [10]:
# NOW-OLD FUNCTION
# def GetLoss(_mst_adj_graph, _mask_de_correct_edges, _negLogLikelies):
#     _negLogLikelies = _negLogLikelies.copy()
#     _negLogLikelies[~_mst_adj_graph] = 0
#     _negLogLikelies[~_mask_de_correct_edges] *= -1 # BAKA!!! Check before you try to fix this again
#     return np.sum(_negLogLikelies)    

# NEW FUNCTION
def GetLoss(_mst_adj_graph, _mask_de_correct_edges, _WScalarMat):
    _WScalarMat = _WScalarMat.copy()
    _WScalarMat[_mst_adj_graph&(~_mask_de_correct_edges)] *= -1 # BAKA!!! Check before you try to fix this again
    _WScalarMat[~_mst_adj_graph] = 0
    return np.sum(_WScalarMat)

In [40]:
class Trainer:
    def __init__(self):
        self.hidden_layer_size = 400
        self._edge_vector_dim = WD._edge_vector_dim
        self._full_cnglist = list(WD.mat_cngCount_1D)
        self.neuralnet = NN(self._edge_vector_dim, self.hidden_layer_size, outer_relu=True)
        self.history = defaultdict(lambda: list())
        
    def Reset(self):
        self.neuralnet = NN(self._edge_vector_dim, self.hidden_layer_size)
        self.history = defaultdict(lambda: list())
        
    def Save(self, filename):
        #pickle.dump({'nnet': self.neuralnet, 'history': dict(self.history)}, open(filename, 'wb'))
        return
        
    
    def Load(self, filename):
        o = pickle.load(open(filename, 'rb'))
        self.neuralnet = o['nnet']
        self.history = defaultdict(lambda: list(), o['history'])    
        
    def Test(self, sentenceObj, dcsObj):
        neuralnet = self.neuralnet
        minScore = np.inf
        minMst = None
        try:
            (nodelist, nodelist_correct, _) = GetTrainingKit(sentenceObj, dcsObj)
            # nodelist = GetNodes(sentenceObj)
        except IndexError:
            print('\x1b[31mError with {} \x1b[0m'.format(sentenceObj.sent_id))
            return (0, 0, 0)
            
        conflicts_Dict = Get_Conflicts(nodelist)
        conflicts_Dict_correct = Get_Conflicts(nodelist_correct)
        
        featVMat = Get_Feat_Vec_Matrix(nodelist, conflicts_Dict)
        featVMat_correct = Get_Feat_Vec_Matrix(nodelist_correct, conflicts_Dict_correct)
        
        if not self.neuralnet.outer_relu:
            (WScalarMat, SigmoidGateOutput) = Get_W_Scalar_Matrix_from_FeatVect_Matrix(featVMat, nodelist, conflicts_Dict, neuralnet)
        else:
            WScalarMat = Get_W_Scalar_Matrix_from_FeatVect_Matrix(featVMat, nodelist, conflicts_Dict, neuralnet)
        # print(WScalarMat)
        # Get all MST
        for source in range(len(nodelist)):
            (mst_nodes, mst_adj_graph) = MST(nodelist, WScalarMat, conflicts_Dict, source)
            # print('.', end = '')
            score = GetMSTWeight(mst_nodes, WScalarMat)
            if(score < minScore):
                minScore = score
                minMst = mst_nodes
        dcsLemmas = [[rom_slp(l) for l in arr]for arr in dcsObj.lemmas]
        word_match = 0
        lemma_match = 0
        n_output_nodes = 0
        for chunk_id, wdSplit in minMst.items():
            for wd in wdSplit:
                n_output_nodes += 1
                # Match lemma
                search_result = [i for i, j in enumerate(dcsLemmas[chunk_id]) if j == wd.lemma]
                if len(search_result) > 0:
                    lemma_match += 1
                # Match CNG
                for i in search_result:
                    if(dcsObj.cng[chunk_id][i] == str(wd.cng)):
                        word_match += 1
                        # print(wd.lemma, wd.cng)
                        break
        dcsLemmas = [l for arr in dcsObj.lemmas for l in arr]
#         print('\nFull Match: {}, Partial Match: {}, OutOf {}, NodeCount: {}, '.\
#               format(word_match, lemma_match, len(dcsLemmas), len(nodelist)))
        return (word_match, lemma_match, len(dcsLemmas), n_output_nodes)
    
    def Train(self, sentenceObj, dcsObj):
        # Hyperparameter for hinge loss: m
        M_hinge = 10
        try:
            (nodelist, nodelist_correct, nodelist_to_correct_mapping) = GetTrainingKit(sentenceObj, dcsObj)
        except IndexError as e:
            # print('\x1b[31mError with {} \x1b[0m'.format(sentenceObj.sent_id))
            # print(e)
            return
        
        """ CREATE A MASK FOR ALL EDGES BETWEEN CORRECT NODE PAIRS"""
        mask_de_correct_edges = np.ndarray((len(nodelist), len(nodelist)), np.bool)*False
        for n1 in nodelist_to_correct_mapping.values():
            for n2 in nodelist_to_correct_mapping.values():
                if n1 != n2:
                    mask_de_correct_edges[n1, n2] = 1
                    mask_de_correct_edges[n2, n1] = 1
                
        
        """ FOR MST OF GRAPH WITH ONLY CORRECT SET OF NODES """
        if not self.neuralnet.outer_relu:
            (conflicts_Dict_correct, featVMat_correct, WScalarMat_correct, _) = GetGraph(nodelist_correct, self.neuralnet)
        else:
            (conflicts_Dict_correct, featVMat_correct, WScalarMat_correct) = GetGraph(nodelist_correct, self.neuralnet)
        source = 0
        (mst_nodes_correct, mst_adj_graph_correct_0) = MST(nodelist_correct, WScalarMat_correct, conflicts_Dict_correct, source)
        
        
        
        """ FOR ALL POSSIBLE MST FROM THE COMPLETE GRAPH """
        if not self.neuralnet.outer_relu:
            (conflicts_Dict, featVMat, WScalarMat, SigmoidGateOutput) = GetGraph(nodelist, self.neuralnet)
        else:
            (conflicts_Dict, featVMat, WScalarMat) = GetGraph(nodelist, self.neuralnet)
        Total_Loss = 0

        Loss_2 = np.sum(WScalarMat*mask_de_correct_edges - WScalarMat*(~mask_de_correct_edges))
        
        # Convert correct spanning tree graph adj matrix to actual marix dimensions
        # Create full-size adjacency matrix for correct_mst
        nodelen = len(nodelist)
        mst_adj_graph_correct = np.ndarray((nodelen, nodelen), np.bool)*False # T_STAR
        for i in range(mst_adj_graph_correct_0.shape[0]):
            for j in range(mst_adj_graph_correct_0.shape[1]):
                mst_adj_graph_correct[nodelist_to_correct_mapping[i], nodelist_to_correct_mapping[j]] = \
                mst_adj_graph_correct_0[i, j]

        # print(WScalarMat)

        """ For each node - Find MST with that source"""
        dLdOut = np.zeros(WScalarMat.shape)
        
        worst_Tx = None
        worst_graph_dist = 0
        
        for source in range(len(nodelist)):
            (mst_nodes, mst_adj_graph) = MST(nodelist, WScalarMat, conflicts_Dict, source) # T_X
            # print('.', end = '')
            
            graph_dist = np.sum(mst_adj_graph&(~mst_adj_graph_correct))
            if graph_dist > worst_graph_dist:
                worst_graph_dist = graph_dist
                worst_Tx = mst_adj_graph
            # Energy diff should all be negative
            print('Source: [{}], Graph_Dist: {}, Energy Diff: {}'.format(source, graph_dist, np.sum(WScalarMat[mst_adj_graph_correct])\
                                                            - np.sum(WScalarMat[mst_adj_graph])))

            """ Gradient Descent"""
            # Total_Loss += (W_star - GetMSTWeight(mst_nodes, WScalarMat))
            Total_Loss += GetLoss(mst_adj_graph, mask_de_correct_edges, WScalarMat)
            # dLdOut_inner = 1 / WScalarMat
            
            if not self.neuralnet.outer_relu:
                # For new loss function with sigmoid on neuralnet
                dLdOut_inner = (1 - SigmoidGateOutput)
                dLdOut_inner[~mst_adj_graph] = 0 # Edges not in the current tree
                dLdOut_inner[mst_adj_graph^mask_de_correct_edges] = -dLdOut_inner[mst_adj_graph^mask_de_correct_edges]

                dLdOut += dLdOut_inner
            else:
                # For new loss function with ReLU on neuralnet
                dLdOut_inner = np.zeros_like(WScalarMat)
                dLdOut_inner[mst_adj_graph&mask_de_correct_edges] = 1 # Edges in the current tree & correct Graph
                dLdOut_inner[mst_adj_graph&(~mask_de_correct_edges)] = -1

                dLdOut += dLdOut_inner
#         print('dLdOut: ')
#         print(dLdOut)
        self.neuralnet.Back_Prop(dLdOut/len(nodelist), len(nodelist), featVMat)       
        Total_Loss /= len(nodelist)
        self.history[sentenceObj.sent_id].append(Total_Loss)
#         print("\nFileKey: %s, Loss: %6.3f, Loss2: %6.3f" % (sentenceObj.sent_id, Total_Loss, Loss_2))

In [25]:
# trainer.neuralnet.U

In [41]:
trainer = None
def InitModule(_matDB):
    global WD, trainer
    WD.word_definite_extInit(_matDB)
    trainer = Trainer()
InitModule(matDB)
# trainer.neuralnet = pickle.load(open('outputs/train_nnet_t9543323920940.p', 'rb'))['nnet']

443


In [43]:
"""
################################################################################################
################################################################################################
################################################################################################
"""
# if __name__ == '__main__':
# loaded_SKT = pickle.load(open('../Simultaneous_CompatSKT_10K.p', 'rb'))
# loaded_DCS = pickle.load(open('../Simultaneous_DCS_10K.p', 'rb'))
# main(loaded_SKT, loaded_DCS)
np.set_printoptions(suppress=False)
trainer.Train(loaded_SKT['170095.p2'], loaded_DCS['170095.p2'])
# print ("Not Implemented")

Source: [0], Graph_Dist: 2, Energy Diff: -8.608621733835015
Source: [1], Graph_Dist: 3, Energy Diff: -8.608621733835015
Source: [2], Graph_Dist: 5, Energy Diff: -9.018955774620913
Source: [3], Graph_Dist: 3, Energy Diff: -8.915687265800905
Source: [4], Graph_Dist: 5, Energy Diff: -9.175242338592206
Source: [5], Graph_Dist: 6, Energy Diff: -9.312418128449828
Source: [6], Graph_Dist: 6, Energy Diff: -9.489514588894828
Source: [7], Graph_Dist: 5, Energy Diff: -9.344666617665844
Source: [8], Graph_Dist: 5, Energy Diff: -10.494568744842162
Source: [9], Graph_Dist: 4, Energy Diff: -9.245994239335822
Source: [10], Graph_Dist: 5, Energy Diff: -10.494568744842162
Source: [11], Graph_Dist: 8, Energy Diff: -11.371113332403375
Source: [12], Graph_Dist: 8, Energy Diff: -11.371113332403375
Source: [13], Graph_Dist: 5, Energy Diff: -9.981033898879666
Source: [14], Graph_Dist: 5, Energy Diff: -9.980735994767706
Source: [15], Graph_Dist: 4, Energy Diff: -9.981034635292026
Source: [16], Graph_Dist: 5, E

In [176]:
# Complete Training
train(loaded_SKT, loaded_DCS, 200)
playBeep()

Batch:  0
['141141.p2', '206744.p2', '30352.p2', '43009.p2', '5774.p2', '205691.p2', '216192.p2', '352667.p2', '59768.p2', '35561.p2', '433461.p2', '54567.p2', '165647.p2', '32144.p2', '410602.p2']
ITERATION IN 0
Max(delW):   0.000107	Max(delU):   0.000002
Max(delW):   0.000052	Max(delU):   0.000000
Max(delW):   0.000040	Max(delU):   0.000000
Max(delW):   0.000098	Max(delU):   0.000002
Max(delW):   0.000102	Max(delU):   0.000002
Max(delW):   0.000191	Max(delU):   0.000005
Max(delW):   0.000201	Max(delU):   0.000007
Max(delW):   0.000108	Max(delU):   0.000002
Max(delW):   0.000199	Max(delU):   0.000007
Max(delW):   0.000105	Max(delU):   0.000001
Max(delW):   0.000031	Max(delU):   0.000001
Max(delW):   0.000060	Max(delU):   0.000002
Max(delW):   0.000057	Max(delU):   0.000001
Max(delW):   0.000047	Max(delU):   0.000001
Max(delW):   0.000162	Max(delU):   0.000007
ITERATION IN 1
Max(delW):   0.000083	Max(delU):   0.000003
Max(delW):   0.000135	Max(delU):   0.000005
Max(delW):   0.000023	Ma

In [177]:
# Test on training set
test(loaded_SKT, loaded_DCS, n_testSet=200)
playBeep()

0  Checkpoint... 
100  Checkpoint... 
Micro Recall[lemma]: 79.50581395348837
Micro Recall[word]: 65.40697674418605
Micro Precision[lemma]: 66.82956627978008
Micro Precision[word]: 54.97861942577887


In [171]:
# POST TEST 1K SET
test(loaded_SKT, loaded_DCS, n_testSet = 1000)
playBeep()

0  Checkpoint... 
100  Checkpoint... 
200  Checkpoint... 
300  Checkpoint... 
400  Checkpoint... 
500  Checkpoint... 
600  Checkpoint... 
700  Checkpoint... 
800  Checkpoint... 
900  Checkpoint... 
Micro Recall[lemma]: 80.03557136505114
Micro Recall[word]: 65.95523936564399
Micro Precision[lemma]: 68.09583858764186
Micro Precision[word]: 56.11601513240858


In [178]:
trainer.neuralnet.W

array([[ 0.01404357,  0.01209347,  0.01794848, ...,  0.0234456 ,
         0.01301233,  0.01970181],
       [ 0.00065902, -0.0048814 , -0.0002981 , ..., -0.00220287,
        -0.00015808,  0.00063262],
       [ 0.01546943,  0.01367023,  0.00765777, ...,  0.01640052,
         0.0084043 ,  0.00637307],
       ..., 
       [-0.01056539, -0.0076947 ,  0.00028994, ..., -0.01054845,
         0.00818368, -0.00032479],
       [ 0.03287455,  0.02965553,  0.01249562, ...,  0.03242457,
         0.03653697,  0.02639865],
       [-0.00787064, -0.00180258,  0.00868818, ...,  0.00432711,
        -0.00838297, -0.00391697]])

In [179]:
trainer.neuralnet.U

array([[  4.17664262e-02],
       [ -8.20553864e-02],
       [  4.08658124e-02],
       [ -6.72224475e-03],
       [  1.15924568e-01],
       [ -3.15936676e-02],
       [ -9.74782509e-02],
       [  1.07443571e-02],
       [ -1.15589201e-02],
       [  2.70378160e-02],
       [ -2.49157920e-02],
       [  6.55233639e-02],
       [  8.34945162e-02],
       [  1.60358622e-02],
       [ -5.98017076e-02],
       [ -1.07353618e-03],
       [  4.22504880e-02],
       [ -3.39831804e-02],
       [  1.03187780e-01],
       [  5.59358100e-02],
       [  7.87936091e-02],
       [  1.04397166e-01],
       [  6.42663197e-02],
       [  9.31736401e-02],
       [  5.63897290e-02],
       [ -2.55412566e-02],
       [ -2.53496873e-02],
       [  4.95694320e-02],
       [  2.28684778e-02],
       [ -9.88818448e-02],
       [ -7.04676697e-02],
       [  1.48144030e-01],
       [ -7.03961634e-02],
       [  5.46811899e-02],
       [  3.87931369e-02],
       [ -4.84227028e-02],
       [  5.56228454e-02],
 

3